In [0]:
%run /Workspace/Users/themallpocaws@inteltion.com/Data-Integration-Scenarios/00_common/common_function

In [0]:
CREATE TABLE IF NOT EXISTS POC_DWH.POC.TH_WEATHER_FORECAST (
LATITUDE numeric(12,2),
LONGITUDE numeric(12,2),
FORECAST_DATE DATE,
FORECAST_TIME TIMESTAMP,
FORECAST_TEMP numeric(12,2),
FORECAST_TEMP_UNIT STRING
);

In [0]:
%python
import requests
import json  # Import JSON module

# Define the API URL
api_url = "https://api.open-meteo.com/v1/forecast?latitude=15.5&longitude=101&hourly=temperature_2m&timezone=Asia%2FBangkok&forecast_days=1"

# Fetch data from the API
response = requests.get(api_url)

# Convert response JSON to a string with double quotes
formatted_json = json.dumps(response.json(), indent=2)

# Print properly formatted JSON
print(formatted_json)


In [0]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, to_date, to_timestamp
from pyspark.sql.types import DecimalType
from datetime import datetime
import pytz
batch_status=set_process_success()
try:
    # Initialize Spark session
    spark = SparkSession.builder.getOrCreate()
    asdsad
    thai_tz = pytz.timezone('Asia/Bangkok')
    today_date = datetime.now(thai_tz).strftime('%Y-%m-%d')

    # API response data
    data = response.json()

    # Extract values
    latitude = data["latitude"]
    longitude = data["longitude"]
    forecast_times = data["hourly"]["time"]
    forecast_temps = data["hourly"]["temperature_2m"]
    forecast_temp_unit = "Celsius"

    # Create a list of tuples for DataFrame
    rows = [
        (latitude, longitude, dt[:10], dt, temp, forecast_temp_unit)
        for dt, temp in zip(forecast_times, forecast_temps)
    ]

    # Define schema and create DataFrame
    schema = ["LATITUDE", "LONGITUDE", "FORECAST_DATE", "FORECAST_TIME", "FORECAST_TEMP", "FORECAST_TEMP_UNIT"]
    df = spark.createDataFrame(rows, schema)
    # Transform date and timestamp columns
    df = df.withColumn("LATITUDE", col("LATITUDE").cast(DecimalType(12,2))) \
       .withColumn("LONGITUDE", col("LONGITUDE").cast(DecimalType(12,2))) \
       .withColumn("FORECAST_DATE", to_date(col("FORECAST_DATE"), "yyyy-MM-dd")) \
       .withColumn("FORECAST_TIME", to_timestamp(col("FORECAST_TIME"), "yyyy-MM-dd'T'HH:mm")) \
       .withColumn("FORECAST_TEMP", col("FORECAST_TEMP").cast(DecimalType(12,2)))
    # Write to Databricks SQL table
    df.write.mode("sdsdsdoverwrite") \
    .saveAsTable("POC_DWH.POC.TH_WEATHER_FORECAST")

except Exception as err:
    batch_status=set_process_failed(err)

dbutils.notebook.exit(batch_status)  